In [1]:
## Here we are importing packages in python 

import pandas as pd
import numpy as np
import re
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, PCA, TruncatedSVD, FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score

In [17]:
## Here we are assigning are data to mydata to make our lives easier. You can name it whatever you would like!
df = pd.read_csv('AdoptAPixel3km2020_GO_LandCover.csv')
df

,OBJECTID,landcoversLandCoverId,landcoversDataSource,landcoversMeasuredAt,siteName,landcoversMeasurementLatitude,landcoversMeasurementLongitude,elevation,landcoversMeasurementElevation,landcoversMucCode,...,landcoversSnowIce,landcoversSouthClassifications,landcoversSouthExtraData,landcoversSouthPhotoUrl,landcoversStandingWater,landcoversUpwardExtraData,landcoversUpwardPhotoUrl,landcoversWestClassifications,landcoversWestExtraData,landcoversWestPhotoUrl
0,1,32868,GLOBE Observer App,6/17/20 22:48,14RPU075517,30.292400,-97.881699,234.500000,223.100006,NaN,...,False,NaN,"(source: app, (compassData.horizon: -5.8276998...",https://data.globe.gov/system/photos/2020/06/1...,False,"(source: app, (compassData.horizon: 47.4946526...",https://data.globe.gov/system/photos/2020/06/1...,NaN,"(source: app, (compassData.horizon: 9.15576473...",https://data.globe.gov/system/photos/2020/06/1...
1,2,35334,GLOBE Observer App,7/21/20 22:45,14RPU075517,30.292601,-97.881798,234.500000,221.699997,M02,...,False,"60% MUC 02 (b) [Trees, Closely Spaced, Deciduo...","(source: app, (compassData.horizon: -38.862310...",https://data.globe.gov/system/photos/2020/07/2...,False,"(source: app, (compassData.horizon: 45.3075218...",https://data.globe.gov/system/photos/2020/07/2...,"70% MUC 02 (b) [Trees, Closely Spaced, Deciduo...","(source: app, (compassData.horizon: -1.4739523...",https://data.globe.gov/system/photos/2020/07/2...
2,3,32870,GLOBE Observer App,6/20/20 0:19,14RPU059516,30.292299,-97.898399,227.000000,232.800003,NaN,...,False,NaN,"(source: app, (compassData.horizon: -5.5221615...",https://data.globe.gov/system/photos/2020/06/2...,False,"(source: app, (compassData.horizon: 88.8598269...",https://data.globe.gov/system/photos/2020/06/2...,NaN,"(source: app, (compassData.horizon: -2.0383559...",https://data.globe.gov/system/photos/2020/06/2...
3,4,32872,GLOBE Observer App,6/20/20 0:51,14RPU074522,30.297001,-97.883003,238.699997,225.899994,NaN,...,False,NaN,"(source: app, (compassData.horizon: -4.3381124...",https://data.globe.gov/system/photos/2020/06/2...,False,"(source: app, (compassData.horizon: 87.3564243...",https://data.globe.gov/system/photos/2020/06/2...,NaN,"(source: app, (compassData.horizon: 0.59026524...",https://data.globe.gov/system/photos/2020/06/2...
4,5,32873,GLOBE Observer App,6/20/20 1:19,14RPU069532,30.305901,-97.887703,269.299988,273.100006,NaN,...,False,NaN,"(source: app, (compassData.horizon: -4.9155761...",https://data.globe.gov/system/photos/2020/06/2...,False,"(source: app, (compassData.horizon: 32.7417718...",https://data.globe.gov/system/photos/2020/06/2...,NaN,"(source: app, (compassData.horizon: -15.324676...",https://data.globe.gov/system/photos/2020/06/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1043,34858,GLOBE Observer App,6/22/20 23:43,18TWK487741,40.416401,-74.425598,34.400002,33.599998,NaN,...,False,NaN,NaN,NaN,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...
1043,1044,34862,GLOBE Observer App,6/22/20 23:47,18TWK488744,40.419399,-74.423698,35.900002,36.599998,NaN,...,False,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,NaN,NaN
1044,1045,34863,GLOBE Observer App,6/22/20 23:51,18TWK486751,40.425201,-74.426697,30.200001,29.900000,NaN,...,False,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...
1045,1046,34864,GLOBE Observer App,6/22/20 23:54,18TWK486753,40.427799,-74.426903,30.700001,30.700001,NaN,...,False,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2

In [26]:
suspect_df = df.groupby(by=['landcoversMeasuredAt','landcoversMeasurementLatitude','landcoversMeasurementLongitude']).filter(lambda x: len(x) > threshold)

# writes the entries to a CSV
suspect_df.to_csv("Possible Events.csv")       

# displays the suspected entries
suspect_df

,OBJECTID,landcoversLandCoverId,landcoversDataSource,landcoversMeasuredAt,siteName,landcoversMeasurementLatitude,landcoversMeasurementLongitude,elevation,landcoversMeasurementElevation,landcoversMucCode,...,landcoversSnowIce,landcoversSouthClassifications,landcoversSouthExtraData,landcoversSouthPhotoUrl,landcoversStandingWater,landcoversUpwardExtraData,landcoversUpwardPhotoUrl,landcoversWestClassifications,landcoversWestExtraData,landcoversWestPhotoUrl


In [27]:
suspect_mask = df.isin(suspect_df)

df = df[~suspect_mask].dropna(how = "all")

df.to_csv("Clean CSV.csv")

# display's cleaned data
df

,OBJECTID,landcoversLandCoverId,landcoversDataSource,landcoversMeasuredAt,siteName,landcoversMeasurementLatitude,landcoversMeasurementLongitude,elevation,landcoversMeasurementElevation,landcoversMucCode,...,landcoversSnowIce,landcoversSouthClassifications,landcoversSouthExtraData,landcoversSouthPhotoUrl,landcoversStandingWater,landcoversUpwardExtraData,landcoversUpwardPhotoUrl,landcoversWestClassifications,landcoversWestExtraData,landcoversWestPhotoUrl
0,1,32868,GLOBE Observer App,6/17/20 22:48,14RPU075517,30.292400,-97.881699,234.500000,223.100006,NaN,...,False,NaN,"(source: app, (compassData.horizon: -5.8276998...",https://data.globe.gov/system/photos/2020/06/1...,False,"(source: app, (compassData.horizon: 47.4946526...",https://data.globe.gov/system/photos/2020/06/1...,NaN,"(source: app, (compassData.horizon: 9.15576473...",https://data.globe.gov/system/photos/2020/06/1...
1,2,35334,GLOBE Observer App,7/21/20 22:45,14RPU075517,30.292601,-97.881798,234.500000,221.699997,M02,...,False,"60% MUC 02 (b) [Trees, Closely Spaced, Deciduo...","(source: app, (compassData.horizon: -38.862310...",https://data.globe.gov/system/photos/2020/07/2...,False,"(source: app, (compassData.horizon: 45.3075218...",https://data.globe.gov/system/photos/2020/07/2...,"70% MUC 02 (b) [Trees, Closely Spaced, Deciduo...","(source: app, (compassData.horizon: -1.4739523...",https://data.globe.gov/system/photos/2020/07/2...
2,3,32870,GLOBE Observer App,6/20/20 0:19,14RPU059516,30.292299,-97.898399,227.000000,232.800003,NaN,...,False,NaN,"(source: app, (compassData.horizon: -5.5221615...",https://data.globe.gov/system/photos/2020/06/2...,False,"(source: app, (compassData.horizon: 88.8598269...",https://data.globe.gov/system/photos/2020/06/2...,NaN,"(source: app, (compassData.horizon: -2.0383559...",https://data.globe.gov/system/photos/2020/06/2...
3,4,32872,GLOBE Observer App,6/20/20 0:51,14RPU074522,30.297001,-97.883003,238.699997,225.899994,NaN,...,False,NaN,"(source: app, (compassData.horizon: -4.3381124...",https://data.globe.gov/system/photos/2020/06/2...,False,"(source: app, (compassData.horizon: 87.3564243...",https://data.globe.gov/system/photos/2020/06/2...,NaN,"(source: app, (compassData.horizon: 0.59026524...",https://data.globe.gov/system/photos/2020/06/2...
4,5,32873,GLOBE Observer App,6/20/20 1:19,14RPU069532,30.305901,-97.887703,269.299988,273.100006,NaN,...,False,NaN,"(source: app, (compassData.horizon: -4.9155761...",https://data.globe.gov/system/photos/2020/06/2...,False,"(source: app, (compassData.horizon: 32.7417718...",https://data.globe.gov/system/photos/2020/06/2...,NaN,"(source: app, (compassData.horizon: -15.324676...",https://data.globe.gov/system/photos/2020/06/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,1043,34858,GLOBE Observer App,6/22/20 23:43,18TWK487741,40.416401,-74.425598,34.400002,33.599998,NaN,...,False,NaN,NaN,NaN,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...
1043,1044,34862,GLOBE Observer App,6/22/20 23:47,18TWK488744,40.419399,-74.423698,35.900002,36.599998,NaN,...,False,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,NaN,NaN
1044,1045,34863,GLOBE Observer App,6/22/20 23:51,18TWK486751,40.425201,-74.426697,30.200001,29.900000,NaN,...,False,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...
1045,1046,34864,GLOBE Observer App,6/22/20 23:54,18TWK486753,40.427799,-74.426903,30.700001,30.700001,NaN,...,False,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,False,(source: camera),https://data.globe.gov/system/photos/2020/06/2...,NaN,(source: camera),https://data.globe.gov/system/photos/2020/06/2

In [29]:

suspect_groups = suspect_df.groupby(by = ['landcoversMeasuredAt','landcoversMeasuredAt','landcoversMeasurementElevation','landcoversMeasurementLatitude','landcoversMeasurementLongitude'])


In [30]:
# create a groups dataset
groups_list = []
groups_data = pd.DataFrame()
for group, df in suspect_groups:
    groups_list.append(str(group))
    new_df = df.copy(True)
    new_df["Group Name"] = str(group)
    groups_data = groups_data.append(new_df, ignore_index = True)
groups_df= pd.DataFrame(groups_list, columns=["Group"])
groups_data.to_csv("Group Data.csv")
groups_df

,Group
